# Imports

In [ ]:
from matplotlib import pyplot as plt
import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn import tree
import random
from collections import Counter
import os
import math


# Feature validation

In [ ]:
mev_features =[ 'sumMT', 'met', 'meff', 'ht', "jet_1_pt", "jet_1_mtMet",  "jet_2_pt", "jet_2_mtMet",  "tau_1_pt", "tau_1_mtMet" ]

b_weights   = pd.concat(
    [background[0]['lumiweight']*background[0]['mcEventWeight']*background[0]['pileupweight'],
     background[1]['lumiweight']*background[1]['mcEventWeight']*background[1]['pileupweight'],
     background[2]['lumiweight']*background[2]['mcEventWeight']*background[2]['pileupweight'],
     background[3]['lumiweight']*background[3]['mcEventWeight']*background[3]['pileupweight'],
     background[4]['lumiweight']*background[4]['mcEventWeight']*background[4]['pileupweight'],
     background[5]['lumiweight']*background[5]['mcEventWeight']*background[5]['pileupweight'],
     background[6]['lumiweight']*background[6]['mcEventWeight']*background[6]['pileupweight'],
     background[7]['lumiweight']*background[7]['mcEventWeight']*background[7]['pileupweight'],
     background[8]['lumiweight']*background[8]['mcEventWeight']*background[8]['pileupweight'],
     background[9]['lumiweight']*background[9]['mcEventWeight']*background[9]['pileupweight'],
     background[10]['lumiweight']*background[10]['mcEventWeight']*background[10]['pileupweight']
    ])

data_weights = np.array(data['lumiweight']*data['mcEventWeight']*data['pileupweight'])
signal_weights = np.array(signal['lumiweight']*signal['mcEventWeight']*signal['pileupweight'])

In [ ]:

#
#
#    Some of the code genereating feature validation plots has been 
#    inspired by a previous master thesis by Øivind Birkeland.
#    https://drive.google.com/drive/folders/176ZulAphX8f10QznOJWATfc2Icssu5Zl   
#
#


def plot_validation(feature_name, x_start, x_end, bins, weighted = True, limit = True): 
    concat_background = pd.concat (
        [background[0][feature_name],
         background[1][feature_name],
         background[2][feature_name],
         background[3][feature_name],
         background[4][feature_name],
         background[5][feature_name],
         background[6][feature_name],
         background[7][feature_name],
         background[8][feature_name],
         background[9][feature_name],
         background[10][feature_name]
        ]
    )


    b_weights   = pd.concat(
      [background[0]['lumiweight']*background[0]['mcEventWeight']*background[0]['pileupweight'],
      background[1]['lumiweight']*background[1]['mcEventWeight']*background[1]['pileupweight'],
      background[2]['lumiweight']*background[2]['mcEventWeight']*background[2]['pileupweight'],
      background[3]['lumiweight']*background[3]['mcEventWeight']*background[3]['pileupweight'],
      background[4]['lumiweight']*background[4]['mcEventWeight']*background[4]['pileupweight'],
      background[5]['lumiweight']*background[5]['mcEventWeight']*background[5]['pileupweight'],
      background[6]['lumiweight']*background[6]['mcEventWeight']*background[6]['pileupweight'],
      background[7]['lumiweight']*background[7]['mcEventWeight']*background[7]['pileupweight'],
      background[8]['lumiweight']*background[8]['mcEventWeight']*background[8]['pileupweight'],
      background[9]['lumiweight']*background[9]['mcEventWeight']*background[9]['pileupweight'],
      background[10]['lumiweight']*background[10]['mcEventWeight']*background[10]['pileupweight']
      ])

    background_full = [background[0][feature_name], background[1][feature_name], background[2][feature_name], background[3][feature_name],
          background[4][feature_name], background[5][feature_name], background[6][feature_name], background[7][feature_name],
          background[8][feature_name], background[9][feature_name], background[10][feature_name]]


    bins = np.linspace(data[feature_name].min(),data[feature_name].max(), bins)
    bin_centres = (bins[:-1] + bins[1:])/2
    colors = ['royalblue', 'dodgerblue', 'steelblue', 'skyblue', 'firebrick', 'indianred', 'salmon', 'forestgreen', 'darkgreen', 'mediumseagreen', 'darkslategrey']
    colors = ['#003f5c', '#2f4b7c', '#665191', '#a05195', '#d45087', '#f95d6a', '#ff7c43', '#ffa600', '#D9B000', '#CCB311', '#99BF22']


    mc_total_hist = np.histogram(concat_background, bins, weights=b_weights)[0]
    mc_total_err = np.sqrt(np.histogram(concat_background, bins, weights=b_weights**2)[0])
    data_hist = np.histogram(data[feature_name], bins, weights=data_weights)[0]
    data_err = np.sqrt(np.histogram(data[feature_name], bins, weights=data_weights**2)[0])
    data_mc = data_hist / mc_total_hist 
    data_mc_err = np.sqrt( (data_err**2 / data_hist**2 + mc_total_err**2 / mc_total_hist**2) * data_mc**2 )


    plt.clf()
    fig = plt.figure(figsize=(16, 9))
    ax_1, ax_2 = fig.subplots(2, 1, gridspec_kw={'height_ratios': [4, 1]})

    if limit:
      ax_1.set_xlim(x_start, x_end)
    ax_1.set_yscale('log')
    ax_1.set_ylim(1e-1, 1e6)
    ax_1.set_ylabel('Events per bin', fontsize=11)
    ax_1.ticklabel_format(axis='x', style='plain')
    if feature_name in mev_features: ax_1.set_xlabel(feature_name + " [MeV]" , fontsize=14, labelpad=5)
    else: ax_1.set_xlabel(feature_name, fontsize=14, labelpad=5)



    #MC sets are stacked and plotted
    ax_1.hist(
        background_full,
        weights=b_weights,
        bins=bins,
        color=colors,
        stacked=True,
        label=backgroundLabel
        )
    
    #MC error is plotted
    ax_1.fill_between(
        bin_centres,
        mc_total_hist+mc_total_err,
        mc_total_hist-mc_total_err,
        step="mid",
        alpha=0.4,
        color="gray",
        zorder=100, 
        
        label='MC error'   
    )

    #Signal is plotted // wieght differently if using all datasets?
    ax_1.hist(
        signal[feature_name],
        weights=signal_weights,
        bins=bins,
        color='firebrick',
        histtype=u'step',
        linewidth=1.2,
        label='Signal'
        )

    #Data is plotted as points with errorbars
    ax_1.errorbar(
        bin_centres,
        data_hist,
        yerr=data_err,

        color='black',
        fmt='o',
        label='data'
    )

    ax_1.legend(
      ncol=2,
      edgecolor="black",
      title_fontsize= 13,
      frameon=True,
      fancybox=True,
      framealpha=0.4,
      loc='best'
      )


    ax_2.set_ylim(0.5, 1.5)
    ax_2.set_yticks([0.50, 0.75, 1.00, 1.25, 1.50])
    ax_2.ticklabel_format(axis='x', style='plain')
    ax_2.set_ylabel('Data / Monte Carlo', fontsize=11)

    #The data/MC is plotted with errorbars
    ax_2.axhline(y=1.0, color='#2f4b7c', linestyle='-')
    ax_2.errorbar(bin_centres, data_mc, yerr=data_mc_err, color='black', fmt='o')

    plt.show()

# Custom decision tree

## Implementation


### imports

In [ ]:
import pkg_resources
pkg_resources.require("numpy==1.19.5")
import numpy
import os
from numba import jit
from statsmodels.stats.weightstats import DescrStatsW
import tkinter as tk

### GPU functions

In [ ]:
#
#
#     The following section includes function that are GPU optimized
#
#

@jit(nopython= True, fastmath=True, parallel= True)
def _custom_split_GPU(parent, parent_weights, child, child_weights):
  parent_background_count = 0
  parent_signal_count = 0

  unweighted_parent_background_count = 0
  unweighted_parent_signal_count = 0
  

  for i in range(len(parent)):
    if parent[i] == 1:
      parent_signal_count += parent_weights[i]
      unweighted_parent_signal_count += parent[i]
    else:
      parent_background_count += parent_weights[i]
      unweighted_parent_background_count += parent[i]

  child_background_count = 0
  child_signal_count = 0

  for i in range(len(child)):
    if child[i] == 1:
      child_signal_count += child_weights[i]
    else:
      child_background_count += child_weights[i]
  try:
    measure_children = child_signal_count / math.sqrt( child_signal_count + child_background_count)
    measure_parent = parent_signal_count / math.sqrt( parent_signal_count + parent_background_count)

    # Alternative loss function using Asimov significance
    # s = child_signal_count
    # b = child_background_count
    # measure_children = math.sqrt(2 * ( (s + b) * math.log(1 + (s/b) )  - s ) )

    s = parent_signal_count
    b = parent_background_count
    measure_parent = math.sqrt(2 * ( (s + b) * math.log(1 + (s/b) )  - s ) )
  except:
    return 0, 0,0, 0, 0, 0 ,0

  measure = measure_children - measure_parent 
  return measure, 0, 0, unweighted_parent_signal_count, unweighted_parent_background_count, parent_signal_count, parent_background_count

  



@jit(nopython= True, fastmath=True)
def _custom_split_GPU_sum(parent, parent_weights, child, child_weights, s_s, s_b):

  child_background_count_sum = s_b
  child_signal_count_sum = s_s

  signal_count, background_count, weighted_signal_count, weighted_background_count = 0, 0, 0, 0
  parent_signal_count, parent_background_count, parent_weighted_signal_count, parent_weighted_background_count = 0, 0, 0, 0


  for i in range(len(parent)):
    if parent[i] == 1:
      parent_weighted_signal_count += parent_weights[i]
      parent_signal_count += 1
    else:
      parent_weighted_background_count += parent_weights[i]
      parent_background_count += 1


  for i in range(len(child)):
    if child[i] == 1:
      weighted_signal_count += child_weights[i]
      signal_count += child[i]
    else:
      weighted_background_count += child_weights[i]
      background_count += child[i]

  try:
    child_signal_count_sum += weighted_signal_count
    child_background_count_sum += weighted_background_count

    measure_child =  child_signal_count_sum / math.sqrt( child_signal_count_sum + child_background_count_sum)

    # Alternative loss function using Asimov significance
    # s = child_signal_count_sum
    # b = child_background_count_sum
    # measure_child = math.sqrt(2 * ( (s + b) * math.log(1 + (s/b) )  - s ) )

    if s_s == 0 and s_b == 0:
      measure_parent = 0
    else:
      # Alternative loss function using Asimov significance
      # s = s_s
      # b = s_b
      # measure_parent = math.sqrt(2 * ( (s + b) * math.log(1 + (s/b) )  - s ) )

      measure_parent = s_s / math.sqrt( s_s + s_b)

    measure = measure_child - measure_parent


    return measure, child_signal_count_sum, child_background_count_sum, parent_signal_count, parent_background_count, parent_weighted_signal_count, parent_weighted_background_count
  except:
    return 0, 0, 0, 0, 0, 0, 0


@jit(nopython=True)
def _variance_reduction(parent, l_child, r_child):
    weight_l = len(l_child) / len(parent)
    weight_r = len(r_child) / len(parent)
    reduction = np.var(parent) - (weight_l * np.var(l_child) + weight_r * np.var(r_child))
    return reduction


@jit(nopython=True)
def _calc_var(arr, arr_weights, mean):
    sum = 0
    count = 0 
    
    for i in range(0,len(arr)):
        sum +=  arr_weights[i]*(arr[i] - mean) ** 2
        count += arr_weights[i]
    try:
      return sum / count
    except: 
      return 0

@jit(nopython=True)
def _get_avg(arr, arr_weights):
    sum = 0 
    count = 0
    for i in range(0,len(arr)):
        sum += (arr[i] * arr_weights[i])
        count += arr_weights[i]
    if count ==0:
      return 0
    return sum / count

@jit(nopython=True)
def _sum(arr):
  sum=0
  for i in range(len(arr)):
    sum+= arr[i]
  return sum

@jit(nopython=True)
def _weighted_variance_reduction(l_child, l_weights, r_child, r_weights):

  try:

    l_mean_w = _get_avg(l_child, l_weights)
    l_var = _calc_var(l_child, l_weights, l_mean_w)

    r_mean_w = _get_avg(r_child, r_weights)
    r_var = _calc_var(r_child, r_weights, r_mean_w)

    p = np.append(l_child, r_child, axis=0)
    p_weights = np.append(l_weights, r_weights, axis=0)

    parent_background_count = 0
    parent_signal_count = 0

    parent_unweighted_background_count = 0
    parent_unweighted_signal_count = 0


    for i in range(len(p)):
      if p[i] == 1:
        parent_signal_count += p_weights[i]
        parent_unweighted_signal_count +=1
      else:
       parent_background_count += p_weights[i]
       parent_unweighted_background_count +=1

    p_mean_w = _get_avg(p, p_weights)
    p_var = _calc_var(p, p_weights, p_mean_w)

    if _sum(p_weights) > 0:
      l_size_weight = _sum(l_weights) / _sum(p_weights)
      r_size_weight = _sum(r_weights) / _sum(p_weights)
    else:
      l_size_weight = 0
      r_size_weight: 0

    reduction = p_var - (l_size_weight * l_var + r_size_weight * r_var)
    return reduction, parent_unweighted_signal_count, parent_unweighted_background_count, parent_signal_count, parent_background_count
  except:
    return 0, 0, 0, 0, 0



@jit(nopython= True, fastmath=True, parallel= True)
def _calculate_sensitivity(preds, target, weights):
    s_s = 0
    b_s = 0
    for i in range(0, len(target)) :
        if target[i] == 1 and preds[i] == 1:
            s_s += weights[i]
        if target[i] == 0 and preds[i] == 1:
            b_s += weights[i]

    return s_s, b_s, (s_s / math.sqrt(( b_s + s_s )))

        



### Tree

In [ ]:
#
#
#     The code responsible for generating decision trees is partially based on following material
#     https://github.com/Suji04/ML_from_Scratch/blob/master/decision%20tree%20regression.ipynb
#     
#     An in-depth description of the functions is provided in Section 6.3
#
#


class Node():
    def __init__(self, feature=None, threshold=None, nth_chosen = None, left=None, right=None, var_red=None,
                  s_s = None, b_b = None, w_s_s = None,  w_b_b = None, value=None):
        
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.var_red = var_red
        self.nth_chosen = nth_chosen


        self.value = value



        self.s_s = s_s
        self.b_b = b_b

        self.w_s_s = w_s_s
        self.w_b_b = w_b_b

In [ ]:

class DecisionTreeRegressor():
    def __init__(self, min_samples_split=1000, max_depth=2, features=[], weights=[], use_sum=False, split_type="custom", model_type="regr", min_split_gain=0.0):

        # initialize the root of the tree
        self.root = None

        # stopping conditions
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.features = features
        self.use_sum = use_sum
        self.split_type = split_type
        self.min_split_gain = min_split_gain
        self.leaf_nodes = 0
        self.tree_str = ""
        self.model_type = model_type
        self.feature_importance_dict = {}

    def get_feature_importance_dict(self):
        return self.feature_importance_dict

    def add_to_feature_importance(self, feature, value):
        new_feature_name = feature.replace("_", "\n")
        if new_feature_name in self.feature_importance_dict:
            self.feature_importance_dict[new_feature_name] += value
        else:
            self.feature_importance_dict[new_feature_name] = value

    def build_tree(self, dataset, curr_depth=0, s_s=0, s_b=0, left="root"):
        num_samples = len(dataset)
        weighted_num_samples = dataset['summed_weight'].sum()

        best_split = {}
        if weighted_num_samples >= self.min_samples_split and curr_depth <= self.max_depth:
            best_split, s_s, s_b = self.get_best_split(
                dataset, num_samples, s_s, s_b)

            if not not best_split:
                if best_split["var_red"] > self.min_split_gain:
                    self.add_to_feature_importance(
                        best_split['feature'], best_split['var_red'])
                    # recur right
                    right_subtree = self.build_tree(
                        best_split["dataset_right"], curr_depth+1, s_s, s_b, left="right")
                    # recur left
                    left_subtree = self.build_tree(
                        best_split["dataset_left"], curr_depth+1, s_s, s_b, left="left")
                    # return decision node
                    return Node(best_split["feature"], best_split["threshold"], best_split["nth_chosen"],
                                left_subtree, right_subtree, best_split["var_red"], best_split["s_s"],
                                best_split["b_b"], best_split["w_s_s"], best_split["w_b_b"])

        leaf_value = _get_avg(dataset['data_type'].to_numpy(), dataset['summed_weight'].to_numpy(
        )) if self.model_type == "regr" else self.calculate_classification_leaf_value(left)

        self.leaf_nodes += 1

        leaf_weighted_background_count = 0
        leaf_weighted_signal_count = 0

        for i in range(0, len(dataset)):
            if dataset['data_type'].values[i] == 1:
                leaf_weighted_signal_count += dataset['summed_weight'].values[i]
            else:
                leaf_weighted_background_count += dataset['summed_weight'].values[i]

        return Node(value=leaf_value, w_s_s=leaf_weighted_signal_count, w_b_b=leaf_weighted_background_count)

    def calculate_sensitivity(self, X, target, weights):
        preds = self.predict(X)
        return _calculate_sensitivity(preds, target, weights)

    def get_num_leaf_nodes(self):
        return self.leaf_nodes

    def calculate_classification_leaf_value(self, left):
        return 1 if left == "right" else 0

    def calculate_leaf_value(self, Y):
        val = np.mean(Y)
        return val

    def print_tree(self, tree=None, indent="|"):
        tree_str = ""

        if not tree:
            tree = self.root

        if tree.value is not None:
            temp = ""
            if type(tree.value) is tuple:
                temp = str(tree.value[0]), "w_s_s: ", str(
                    round(tree.w_s_s, 3)), "w_b_b: ", str(round(tree.w_b_b, 3))
            else:
                temp = str(tree.value), "w_s_s: ", str(
                    round(tree.w_s_s, 3)), "w_b_b: ", str(round(tree.w_b_b, 3))
            self.tree_str += str(temp)
            print(temp)

        else:
            print("[", str(tree.feature), "<=", str(round(tree.threshold, 2)), "?", str(round(tree.var_red, 2)), "(", str(tree.nth_chosen), ") " + ("s_s: " + str(
                round(tree.s_s, 2)) + " | w_s_s: " + str(round(tree.w_s_s, 2)) + " | b_b:  " + str(round(tree.b_b, 2)) + " | w_b_b: " + str(round(tree.w_b_b, 2)) + " ]"))
            temp = "[ " + str(tree.feature) + " " + "<=" + " " + str(round(tree.threshold, 2)) + " " + "?" + " " + str(round(tree.var_red, 2)) + " " + "(" + " " + str(tree.nth_chosen) + ") ]" + \
                "s_s: " + str(round(tree.s_s, 2)) + " | w_s_s: " + str(round(tree.w_s_s, 2)) + \
                " | b_b:  " + str(round(tree.b_b, 2)) + \
                " | w_b_b: " + str(round(tree.w_b_b, 2)) + " ]"
            self.tree_str += temp

            temp = "\n%sleft:" % (indent)
            self.tree_str += temp
            print("|")
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + "--")

            temp = "\n%sright:" % (indent)
            self.tree_str += temp
            print("|")
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + "--")

    def return_tree_str(self):
        return self.tree_str

    def get_root(self):
        return self.root

    def fit(self, dataset):
        self.root = self.build_tree(dataset)

    def make_prediction_with_sens(self, x, tree, sens=-999):
        if tree.value != None:
            return (tree.value, sens)

        # checks if the node is one before leaf node
        if tree.left.value != None or tree.right.value != None:
            signal_as_signal = tree.right.w_s_s
            background_as_signal = tree.left.w_s_s

            sens = 0
            try:
								sens = signal_as_signal / math.sqrt(signal_as_signal + background_as_signal )
            except:
                sens = 0
            feature_val = x[tree.feature]

            if feature_val <= tree.threshold:
                return self.make_prediction_with_sens(x, tree.left, sens)
            else:
                return self.make_prediction_with_sens(x, tree.right, sens)

        else:

            feature_val = x[tree.feature]

            if feature_val <= tree.threshold:
                return self.make_prediction_with_sens(x, tree.left)
            else:
                return self.make_prediction_with_sens(x, tree.right)

    def make_prediction(self, x, tree):
        if tree.value != None:
            return tree.value
        feature_val = x[tree.feature]
        if feature_val <= tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)

    def predict(self, df, use_sens=False):
        if not use_sens:
            preds = df.apply(lambda row: self.make_prediction(
                row, self.root), axis=1)
            return preds.to_numpy()

        preds = df.apply(lambda row: self.make_prediction_with_sens(
            row, self.root), axis=1)
        return zip(*preds)


		#
		#
		# 		Code for automatically generating margins between the x-ticks is partly based on the following link:
		#			https://stackoverflow.com/questions/44863375/how-to-change-spacing-between-ticks-in-matplotlib
		#
		#

    def plot_feature_importance(self, save_path):
        feature_importance = self.feature_importance_dict

        features_sorted = dict(
            sorted(feature_importance.items(), key=lambda item: item[1], reverse=True))
        plt.clf()
        plt.title("Decision tree feature importance")

        plt.bar(range(len(features_sorted)), list(
            features_sorted.values()), align='center')
        plt.xticks(range(len(features_sorted)), list(features_sorted.keys()))

        N = len(self.features)
        plt.xticks(range(N))  # add loads of ticks
        plt.grid()

        plt.gca().margins(x=0)
        plt.gcf().canvas.draw()
        tl = plt.gca().get_xticklabels()
        maxsize = max([t.get_window_extent().width for t in tl])
        m = 1.0  # inch margin
        s = maxsize/plt.gcf().dpi*N+2*m
        margin = m/plt.gcf().get_size_inches()[0]

        plt.gcf().subplots_adjust(left=margin, right=1.-margin)
        plt.gcf().set_size_inches(s, plt.gcf().get_size_inches()[1])

        if save_path:
            final_save_path = save_path + "feature-importance.png"
            plt.savefig(final_save_path)

        plt.show()

    def get_best_split(self, dataset, num_samples, s_s, s_b):

        best_split = {}
        new_s_s, curr_s_s = 0, 0
        new_s_b, curr_s_b = 0, 0
        max_all_feature_split_measure = -float("inf")
        leaf_weighted_signal_cound, background_count, weighted_signal_count, weighted_background_count = 0, 0, 0, 0
        counter = 0

        for feature in self.features:
            feature_values = dataset[feature]
            all_thresholds = np.sort(
                np.unique(feature_values.round(decimals=3)))
            all_thresholds = self.get_reduced_threshold_list(
                np.sort(all_thresholds))

            counter = 0

            for threshold in all_thresholds:
                counter += 1
                dataset_left, dataset_right = self.split(
                    dataset, feature, threshold)
                if len(dataset_left) > 0 and len(dataset_right) > 0:
                    p, p_weights = dataset['data_type'].to_numpy(
                    ), dataset['summed_weight'].to_numpy(),
                    r_c, r_c_weights = dataset_right['data_type'].to_numpy(
                    ), dataset_right['summed_weight'].to_numpy()
                    l_c, l_c_weights = dataset_left['data_type'].to_numpy(
                    ), dataset_left['summed_weight'].to_numpy()

                    if self.split_type == "custom":
                        current_feature_measure, curr_s_s, curr_s_b, signal_count, background_count, weighted_signal_count, weighted_background_count = self.custom_split_func_3(
                            p, p_weights, r_c, r_c_weights, l_c, l_c_weights, s_s, s_b)
                    else:
                        current_feature_measure, signal_count, background_count, weighted_signal_count, weighted_background_count = self.variance_reduction(
                            p, l_c, l_c_weights, r_c, r_c_weights)
                        curr_s_s = 0
                        curr_s_b = 0

                    if current_feature_measure > max_all_feature_split_measure:

                        best_split["feature"] = feature
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["var_red"] = current_feature_measure
                        best_split["nth_chosen"] = counter
                        best_split["s_s"] = signal_count
                        best_split["w_s_s"] = weighted_signal_count
                        best_split["b_b"] = background_count
                        best_split["w_b_b"] = weighted_background_count
                        max_all_feature_split_measure = current_feature_measure
                        new_s_s = curr_s_s
                        new_s_b = curr_s_b

        return best_split, new_s_s, new_s_b

    def custom_split_func(self, p, p_weights, r_c, r_c_weights, l_c, l_c_weights, s_s, s_b):
        if not self.use_sum:
            return _custom_split_GPU(p, p_weights, r_c, r_c_weights)
        return _custom_split_GPU_sum(p, p_weights, r_c, r_c_weights, s_s, s_b)

    def custom_split_func_2(self, p, p_weights, r_c, r_c_weights, l_c, l_c_weights, s_s, s_b):
        return _custom_split_GPU_2(r_c, r_c_weights, l_c, l_c_weights)

    def custom_split_func_3(self, p, p_weights, r_c, r_c_weights, l_c, l_c_weights, s_s, s_b):
        return _custom_split_GPU_3(r_c, r_c_weights, l_c, l_c_weights)

    def split(self, dataset, feature, threshold):
        left = dataset[dataset[feature] <= threshold]
        right = dataset[dataset[feature] > threshold]
        return left, right

    def get_reduced_threshold_list(self, possible_thresholds):
        total = len(possible_thresholds)
        reduced_list = []
        for i in np.arange(0.0, 1.0, 0.02):
            reduced_list.append(possible_thresholds[math.floor(total*i)])
        return np.array(reduced_list)

    def variance_reduction(self, parent, l_c, l_c_w, r_c, r_c_w):
        return _weighted_variance_reduction(l_c, l_c_w, r_c, r_c_w)


## Random forest

### Prediction functions for feature validation


In [ ]:

def plot_validation(predicted_data, predicted_data_2, feature_name, x_start, x_end, bins, title, weighted = True, limit = False, save_path = ""): 

    signal = predicted_data.loc[predicted_data['data_type'] == 1]
    signal_weights = signal['summed_weight']

    signal_2 = predicted_data_2.loc[predicted_data_2['data_type'] == 1]
    signal_2_weights = signal_2['summed_weight']

    
    background = [              
                  df_test.loc[df_test['full_data_type'] == 2],
                  df_test.loc[df_test['full_data_type'] == 3],
                  df_test.loc[df_test['full_data_type'] == 4],
                  df_test.loc[df_test['full_data_type'] == 5],
                  df_test.loc[df_test['full_data_type'] == 6],
                  df_test.loc[df_test['full_data_type'] == 7],
                  df_test.loc[df_test['full_data_type'] == 8],
                  df_test.loc[df_test['full_data_type'] == 9],
                  df_test.loc[df_test['full_data_type'] == 10],
                  df_test.loc[df_test['full_data_type'] == 11],
                  df_test.loc[df_test['full_data_type'] == 12]
              ]


    x_weights = [
            background[0]['summed_weight'],
            background[1]['summed_weight'],
            background[2]['summed_weight'],
            background[3]['summed_weight'],
            background[4]['summed_weight'],
            background[5]['summed_weight'],
            background[6]['summed_weight'],
            background[7]['summed_weight'],
            background[8]['summed_weight'],
            background[9]['summed_weight'],
            background[10]['summed_weight']
            ]


            

    x = [background[0][feature_name], background[1][feature_name], background[2][feature_name], background[3][feature_name],
          background[4][feature_name], background[5][feature_name], background[6][feature_name], background[7][feature_name],
          background[8][feature_name], background[9][feature_name], background[10][feature_name]]



    max_value= max(signal[feature_name].max(), signal_2[feature_name].max())
    
    for i in range(0, len(x)):
      max_value = max(max_value, x[i].max())



    
    bins = np.linspace(0, (max_value + max_value * 0.05))
    bin_centres = (bins[:-1] + bins[1:])/2
    colors = ['royalblue', 'dodgerblue', 'steelblue', 'skyblue', 'firebrick', 'indianred', 'salmon', 'forestgreen', 'darkgreen', 'mediumseagreen', 'darkslategrey']


    plt.clf()
    plt.figure(figsize=(16, 9))
    ax_1  = plt.gca()

    if limit:
      ax_1.set_xlim(x_start, x_end)
    ax_1.set_yscale('log')
    ax_1.set_ylim(1e-1, 1e6)
    ax_1.set_ylabel('Events per bin', fontsize=11)
    ax_1.ticklabel_format(axis='x', style='plain')
    ax_1.set_xlabel(feature_name + " " , fontsize=13)
    ax_1.set_title(title)


    ax_1.hist(
        x,
        weights=x_weights,
        bins=bins,
        color=colors,
        stacked=True,
        label=backgroundLabel
        )
    
    ax_1.hist(
        signal[feature_name],
        weights=signal_weights,
        bins=bins,
        color='firebrick',
        histtype=u'step',
        linewidth=1.2,
        label='Signal'
        )


    ax_1.legend(
      ncol=2,
      edgecolor="black",
      title_fontsize= 13,
      frameon=True,
      fancybox=True,
      framealpha=0.4,
      loc='best'
      )

    if save_path != "":
      plt.savefig(save_path)
    plt.show()

In [ ]:
def classify_with_threshold(preds, threshold):
  return [1 if pred > threshold else 0 for pred in preds]

In [ ]:

def predict_with_feature_validation(min_samples_split, max_depth, features, weights, df_train, df_test, save_path, n_estimators=60, train_frac=0.125, max_features="auto"):
    most_important_features = ["jet_1_pt", "sumMT", "tau_1_mtMet",
                               "METoverPtMean", "jet_2_mtMet", "ht", "tau_1_mtMet", "met", "meff"]
    # Variance reduction
    print("------------------------VARIANCE REDUCTION-----------------------")

    var_red_save_path = save_path + "-var-red-"

    regressor_var_red = RandomForest(min_samples_split, max_depth, features, weights,
                                      False, "var_red", n_estimators, model_type="regr", max_features="auto")
    regressor_var_red.fit(df_train, train_frac)
    regressor_var_red.plot_feature_importance(save_path=var_red_save_path)
    regressor_var_red.get_tree_info(save_path=var_red_save_path)

    var_red_preds = regressor_var_red.predict(df_test)

    regressor_var_red_max_sens_threshold = plot_sensitivity(
        df_test, var_red_preds, np.array([]), max_depth, min_samples_split, var_red_save_path)
    regressor_var_red_classification = classify_with_threshold(
        var_red_preds, regressor_var_red_max_sens_threshold)

    df_test['predicted_data_type'] = regressor_var_red_classification

    predicted_as_signal = df_test.loc[df_test['predicted_data_type'] == 1]
    predicted_as_background = df_test.loc[df_test['predicted_data_type'] == 0]

    for feature in most_important_features:
        plot_validation(predicted_as_signal, predicted_as_background,  feature, 18e4, 30e4, 300,
                        title="var-red signal prediction", limit=False, save_path=var_red_save_path + feature + "-signal")
        plot_validation(predicted_as_background, predicted_as_signal, feature, 18e4, 30e4, 300,
                        title="var-red background prediction", limit=False, save_path=var_red_save_path + feature + "-background")

    # Non-sum custom loss funcion

    print("------------------------NON SUM -----------------------")

    non_sum_save_path = save_path + "-non-sum-"

    regressor_non_sum = RandomForest(min_samples_split, max_depth, features, weights,
                                      False, "custom_1", n_estimators, model_type="regr", max_features="auto")
    regressor_non_sum.fit(df_train, train_frac)
    regressor_non_sum.plot_feature_importance(save_path=non_sum_save_path)
    regressor_non_sum.get_tree_info(save_path=non_sum_save_path)

    non_sum_preds = regressor_non_sum.predict(df_test)

    regressor_non_sum_max_sens_threshold = plot_sensitivity(
        df_test, non_sum_preds, np.array([]), max_depth, min_samples_split, non_sum_save_path)
    regressor_non_sum_classification = classify_with_threshold(
        non_sum_preds, regressor_non_sum_max_sens_threshold)

    df_test['predicted_data_type'] = regressor_non_sum_classification

    predicted_as_signal = df_test.loc[df_test['predicted_data_type'] == 1]
    predicted_as_background = df_test.loc[df_test['predicted_data_type'] == 0]

    for feature in most_important_features:
        plot_validation(predicted_as_signal, predicted_as_background, feature, 18e4, 30e4, 300,
                        title="non-sum signal prediction", limit=False,  save_path=non_sum_save_path + feature + "-signal")
        plot_validation(predicted_as_background, predicted_as_signal, feature, 18e4, 30e4, 300,
                        title="non-sum background prediction", limit=False,  save_path=non_sum_save_path + feature + "-background")


    #Custom sum loss function 
    
    print("------------------------ SUM -----------------------")

    sum_save_path = save_path + "-sum-"

    classifier_sum = RandomForest(min_samples_split, max_depth, features, weights,
                                   True, "custom_1", n_estimators, model_type="class", max_features="auto")
    classifier_sum.fit(df_train, train_frac)

    _, sum_preds = classifier_sum.predict_class(df_test)

    classifier_sum.get_tree_info(save_path=sum_save_path)
    classifier_sum.plot_feature_importance(save_path=sum_save_path)

    s_s, b_s, max_sensitivity = _calculate_sensitivity(
        sum_preds, df_test['data_type'].to_numpy(), df_test['summed_weight'].to_numpy())

    print("Signal predicted as Signal", s_s)
    print("Background predicted as Signal", b_s)
    print("Maximum sensitivity", max_sensitivity)
    print()

    df_test['predicted_data_type'] = sum_preds

    predicted_as_signal = df_test.loc[df_test['predicted_data_type'] == 1]
    predicted_as_background = df_test.loc[df_test['predicted_data_type'] == 0]

    for feature in most_important_features:
        plot_validation(predicted_as_signal, predicted_as_background, feature, 18e4, 30e4, 300,
                        title="sum signal prediction", limit=False, save_path=sum_save_path + feature + "-signal")
        plot_validation(predicted_as_background, predicted_as_signal, feature, 18e4, 30e4, 300,
                        title="sum background prediction", limit=False, save_path=sum_save_path + feature + "-background")


### Other helper functions


In [ ]:

def make_and_evalaute_model_RF(min_samples_split, max_depth, features, weights, use_sum, df_train, df_test, split_type, n_estimators =10, train_frac= 0.1, save_path = "", model_type = "regr", use_var_red = True, max_features = "auto", predict_on_train = False):
  print("------------------------ RF REGR------------------------")
  
  save_path +=  model_type + "-"

  model = RandomForest(min_samples_split, max_depth, features,weights, use_sum,  split_type, n_estimators, model_type = "regr", max_features = max_features)
  model.fit(df_train_valid, train_frac)

  all_preds= model.predict(df_test)
  model.get_tree_info(save_path = save_path)

  if predict_on_train:
    train_all_preds = model.predict(df_train)
    model.get_tree_info(save_path =  save_path + "train-")

  model.plot_feature_importance(save_path = save_path)

  if use_var_red:
    print()
    print("------------------------ RF REGR VAR RED ------------------------")

    regressor_sklearn = RandomForest(min_samples_split, max_depth, features,weights, False, "var_red", n_estimators, model_type = model_type, max_features = max_features)
    regressor_sklearn.fit(df_train, train_frac)

    all_predict_sklearn = regressor_sklearn.predict(df_test)
    regressor_sklearn.get_tree_info(save_path =   save_path + "var-red-")
    regressor_sklearn.plot_feature_importance(save_path =   save_path + "var-red-")

    var_red_save_path = save_path + "-var-red"

    
    if predict_on_train:
      train_all_predict_sklearn = regressor_sklearn.predict(df_train)
      regressor_sklearn.get_tree_info(save_path =   save_path + "train-var-red-")
      print("------------------------ TRAIN DATA ------------------------")
      plot_sensitivity(df_train, train_all_preds, train_all_predict_sklearn, max_depth, min_samples_split,  var_red_save_path + "train-")
    print("------------------------ TEST DATA ------------------------")
    return plot_sensitivity(df_test, all_preds, all_predict_sklearn, max_depth, min_samples_split, var_red_save_path)

  if train_preds:

    print("------------------------ TRAIN DATA ------------------------")
    plot_sensitivity(df_train, train_all_preds, np.array([]), max_depth, min_samples_split,  save_path + "train-")

  print("------------------------ TEST DATA ------------------------")
  return plot_sensitivity(df_test, all_preds, np.array([]), max_depth, min_samples_split, save_path)




def make_and_evalaute_model_RF_class(min_samples_split, max_depth, features, weights, use_sum, df_train, df_test, split_type, n_estimators =10, train_frac= 0.1, save_path = "", model_type = "class", use_var_red = True, weighted = "both", max_features = "auto", predict_on_train = False):
  print("------------------------ RF CLASS------------------------")

  save_path +=  model_type + "-"


  model = RandomForest(min_samples_split = min_samples_split, max_depth= max_depth, features = features,weights = ['summed_weight'], use_sum = use_sum,  split_type = "custom_1", n_estimators = n_estimators , model_type = "class", max_features = max_features)
  model.fit(df_train_valid, train_frac)

  all_preds, all_unweighted_preds = model.predict_class(df_test)
  model.plot_feature_importance(save_path = save_path)
  model.get_tree_info(save_path = save_path)

  if predict_on_train:
    all_train_preds, all_unweighted_train_preds = model.predict_class(df_train)
    model.get_tree_info(save_path = save_path + "train-")



  if use_var_red:
  
    regressor_sklearn = RandomForest(min_samples_split = min_samples_split, max_depth= max_depth, features = features,weights = ['summed_weight'], use_sum = False,  split_type = "var_red", n_estimators = n_estimators , model_type = "regr", max_features = max_features)
    regressor_sklearn.fit(df_train, train_frac)

    all_predict_sklearn = regressor_sklearn.predict_regr(df_test)
    
    assess_model_classification(df_test, all_preds, all_predict_sklearn, max_depth, min_samples_split, save_path)
  
  if weighted == True:
    s_s, b_s, max_sensitivity = _calculate_sensitivity(all_preds, df_test['data_type'].to_numpy(), df_test['summed_weight'].to_numpy())

    print("Signal predicted as Signal", s_s)
    print("Background predicted as Signal", b_s)
    print("Maximum sensitivity", max_sensitivity)
    print()
    return round(max_sensitivity,3)
    
  elif weighted == "both":

    s_s, b_s, max_sensitivity = _calculate_sensitivity(all_preds, df_test['data_type'].to_numpy(), df_test['summed_weight'].to_numpy())
    u_s_s, u_b_s, u_max_sensitivity = _calculate_sensitivity(all_unweighted_preds, df_test['data_type'].to_numpy(), df_test['summed_weight'].to_numpy())



    print("Signal predicted as Signal", s_s)
    print("Background predicted as Signal", b_s)
    print("Maximum sensitivity", max_sensitivity)
    print()

    print("Unweighted Signal predicted as Signal", u_s_s)
    print("Unweighted Background predicted as Signal", u_b_s)
    print("Unweighted Maximum sensitivity", u_max_sensitivity)
    print()

    if predict_on_train:
      train_s_s, train_b_s, train_max_sensitivity = _calculate_sensitivity(all_train_preds, df_train['data_type'].to_numpy(), df_train['summed_weight'].to_numpy())
      train_u_s_s, train_u_b_s, train_u_max_sensitivity = _calculate_sensitivity(all_unweighted_train_preds, df_train['data_type'].to_numpy(), df_train['summed_weight'].to_numpy())

      print("Train Signal predicted as Signal", train_s_s)
      print("Train Background predicted as Signal", train_b_s)
      print("Train Maximum sensitivity", train_max_sensitivity)
      print()

      print("Train Unweighted Signal predicted as Signal", train_u_s_s)
      print("Train Unweighted Background predicted as Signal", train_u_b_s)
      print("Train Unweighted Maximum sensitivity", train_u_max_sensitivity)
      print()
    return round(max_sensitivity, 3), round(u_max_sensitivity,3)




### implementation

In [ ]:

class RandomForest():
  def __init__(self, min_samples_split=1000, max_depth=2, features = [], weights = [], use_sum= False,  split_type = "", n_estimators = 20, max_features = "auto", model_type = "regr"):
      self.min_samples_split = min_samples_split
      self.max_depth = max_depth
      self.features = features
      self.use_sum = use_sum
      self.s_s = 0
      self.s_b = 0
      self.n_estimators = n_estimators
      self.tree_models = []
      self.model_sensitivities = []
      self.model_features = []
      self.weights = weights
      self.split_type = split_type
      self.model_type = model_type
      self.max_features = max_features

  def plot_feature_importance(self, save_path):
    feature_importance = self.get_cumulative_feature_importance()

    features_sorted =  dict(sorted(feature_importance.items(), key=lambda item: item[1], reverse = True))
    plt.clf()
    plt.title("Random forest feature importance" )

    plt.bar(range(len(features_sorted)), list(features_sorted.values()), align='center')
    plt.xticks(range(len(features_sorted)), list(features_sorted.keys()))

    N = len(self.features)
    plt.xticks(range(N)) 
    plt.grid()

    plt.gca().margins(x=0)
    plt.gcf().canvas.draw()
    tl = plt.gca().get_xticklabels()
    maxsize = max([t.get_window_extent().width for t in tl])
    m = 1.0 
    s = maxsize/plt.gcf().dpi*N+2*m
    margin = m/plt.gcf().get_size_inches()[0]

    plt.gcf().subplots_adjust(left=margin, right=1.-margin)
    plt.gcf().set_size_inches(s, plt.gcf().get_size_inches()[1])

    if save_path:
      final_save_path = save_path + "feature-importance.png"
      plt.savefig(final_save_path) 

    plt.show()
  

  def get_tree_info(self, save_path = ""):
    with_sensitivities = True if self.split_type == "custom_1" and self.model_type != "regr" else False
    
    if with_sensitivities:
      lowest_sens = 999
      lowest_index = 0
      highest_sens = -999
      highest_index = 0

      output_text = ""

      for i in range(0, len(self.model_sensitivities)):
        if (self.model_sensitivities[i] < lowest_sens):
          lowest_index = i
          lowest_sens = self.model_sensitivities[i]

        if (self.model_sensitivities[i] > highest_sens):
          highest_index = i
          highest_sens = self.model_sensitivities[i]

        temp = "Tree " + str(i) + ": " + "\n" + "Sens: " + str(self.model_sensitivities[i]) + ", features: " + "[" + ' '.join(self.model_features[i])+ "]" + "\n"
        output_text += temp

      temp = "Highest sens tree: " + "\n" + "Sens: " + str(self.model_sensitivities[highest_index]) + ", features: " + "[" + ' '.join(self.model_features[highest_index])+ "]" + "\n"
      output_text += temp

      temp = "Lowest sens tree: " + "\n" + "Sens: " + str(self.model_sensitivities[lowest_index]) + ", features: " + "[" + ' '.join(self.model_features[lowest_index])+ "]" + "\n"
      output_text += temp

    else:
      output_text = ""

      for i in range(0, len(self.model_features)):
        temp = "Tree " + str(i) + ": " + "\n" +  ", features: " + "[" + ' '.join(self.model_features[i])+ "]" + "\n"
        output_text += temp

    if save_path:
      final_save_path = save_path + "individual_trees.txt" 
      f= open(final_save_path,"w+")
      f.write(output_text)
      f.close()
        
    return output_text



  def get_trees(self):
    return self.tree_models

  def get_cumulative_feature_importance(self):
    feature_importance_list = []
    for tree in self.tree_models:
      feature_importance_list.append(Counter(tree.get_feature_importance_dict()))
    cumulative_feature_importance = sum(feature_importance_list, Counter())
    relative_cumulative_feature_importance = {k:v/self.n_estimators for k,v in cumulative_feature_importance.items()}


    for feature in relative_cumulative_feature_importance.keys():
      feature_name = feature.replace("\n", "_")
      relative_cumulative_feature_importance[feature] = relative_cumulative_feature_importance[feature] / sum(x.count(feature_name) for x in self.model_features)

    return relative_cumulative_feature_importance

    
  def get_weigthed_data_sample(self, dataset, frac, background_count):
      signal_data = dataset.loc[dataset['data_type'] == 1]

      data_sample = signal_data.sample(frac = frac, replace = True)

      # Append a proportional amount of each background to the data_sample

      for i in range(0, background_count):
          #background is classified incrementally from 2
          background_id = i+2
          bg_sample = dataset.loc[dataset['full_data_type'] == background_id].sample(frac = frac, replace = True)
          data_sample = data_sample.append(bg_sample)
          
      return shuffle(data_sample)
      

  def fit(self, data, single_tree_data_frac):
    for i in range (0,self.n_estimators):
      if self.max_features == "auto":
        selected_features = random.sample(features, math.ceil(math.sqrt(len(features))))
      else:
        selected_features = random.sample(features, self.max_features)
      data_sample = self.get_weigthed_data_sample(data, single_tree_data_frac, 11) # 11 = number of different data types
      model = DecisionTreeRegressor4(self.min_samples_split, self.max_depth, selected_features, self.weights, self.use_sum, self.split_type, self.model_type)
      model.fit(data_sample)
      self.model_features.append(selected_features)
      self.tree_models.append(model)


  def predict(self, data):
    if self.model_type == "regr":
      return self.predict_regr(data)
    else:
      return self.predict_class(data)

  def predict_regr(self, data):
    predictions = []
    
    for model in self.tree_models:
      tree_preds = model.predict(data[self.features])
      _, _, tree_sensitivity = _calculate_sensitivity(tree_preds, data['data_type'].to_numpy(), data['summed_weight'].to_numpy())
      self.model_sensitivities.append(tree_sensitivity)
      predictions.append(tree_preds)
      
    return np.average(predictions, axis=0)


  def predict_class(self, data):
    self.model_sensitivities = []
    predictions = []
    sensitivities = [] 
    for model in self.tree_models:
      if self.use_sum:
        preds = model.predict(data[self.features])
        _, _, tree_sensitivity = _calculate_sensitivity(preds, data['data_type'].to_numpy(), data['summed_weight'].to_numpy())
        sensitivities.append([tree_sensitivity] * len(preds))
        self.model_sensitivities.append(tree_sensitivity)
        predictions.append(preds)

      else:
        preds, sens = model.predict(data[self.features], True)
        sensitivities.append(sens)
        predictions.append(preds)
      
  
    weighted_average_prediction = np.average(predictions, axis=0, weights = sensitivities) # if 0.5, then n_signal_preds == n_background_preds
    unweighted_average_prediction = np.average(predictions, axis=0) # if 0.5, then n_signal_preds == n_background_preds

    weigted_classifications = [1 if x >0.5 else 0 for x in weighted_average_prediction]
    unweigted_classifications = [1 if x >0.5 else 0 for x in unweighted_average_prediction]

   return weigted_classifications, unweigted_classifications




## Functions used for sensitivity plots and other performance assessment


In [ ]:
 def plot_fluctuations(runs_weighted, runs_unweighted): 
    x = []
    for i in range(0, len(runs_unweighted)):
        x.append(i)
    
    
    if runs_weighted:
      min_value = min(runs_weighted + runs_unweighted)
      max_value = max(runs_weighted + runs_unweighted)
    else:
      min_value = min(runs_unweighted)
      max_value = max(runs_unweighted)

    plt.clf()
    plt.figure(figsize=(12,8))
    plt.ylim(min_value - min_value * 0.3, max_value + max_value * 0.3)
    

    if runs_weighted:
      plt.plot(x, runs_weighted, label = "Custom")
      plt.plot(x, runs_weighted, 'bo')
      plt.plot(x, len(runs_weighted)*[np.mean(runs_weighted)], '--', color= "blue", alpha =0.25, label = "Mean custom")

    plt.plot(x, runs_unweighted, label = "Custom unweighted")
    plt.plot(x, runs_unweighted, 'bo')
    plt.plot(x, len(runs_unweighted)*[np.mean(runs_unweighted)], '--', color= "orange", alpha =0.25, label = "Mean custom, unweighted")


    plt.ylabel("Peak Sensitivity")
    plt.xlabel("nth run")
    plt.title("Model's fluctuations using same hyperparameters \n All dataset, 0.25 train_pct, 30 estimators, weighted RF")
    plt.legend(loc="upper right")
    plt.xticks(x)

    if runs_weighted:
        
      for i_x, i_y in zip(x, runs_weighted):
          plt.text(i_x, i_y +0.1, '{}'.format(i_y))

    for i_x, i_y in zip(x, runs_unweighted):
        plt.text(i_x, i_y - 0.1, '{}'.format(i_y))





In [ ]:

def assess_model_classification(data, preds, var_red_preds, depth, min_split, save_path = ""):
  use_var_red = True if var_red_preds.size else False

  s_s, b_s, max_sensitivity = _calculate_sensitivity(preds, data['data_type'].to_numpy(), data['summed_weight'].to_numpy())
  thresholds_vr, sensitivity_measures_vr, max_sensitivity_vr, counts_vr, _, _ = get_sensitivity_values(data, var_red_preds, True)


  rows = ["Custom Test", "SK Test"]
  columns = ["Signal as Signal", "Signal as Background", "Background as Background", "Background as Signal"]
  data = {
      'w_s_s' : [round(s_s,0), counts_vr[0]],
      'w_b_s' : [round(b_s,0), counts_vr[3]],
      'max_sens' : [max_sensitivity, max_sensitivity_vr]
  }

  df = pd.DataFrame(data=data)

  plt.clf()
  plt.figure(figsize=(12,8))
  ylabel = "s/" +  u"\u221A" + "(s+b)"
  plt.ylabel(ylabel)
  plt.xlabel("Model Output")
  plt.title("Custom Maximum Sensitivity: " +  str(round(max_sensitivity, 3))  + "\nVar-Red Maximum Sensitivity: " +  str(round(max_sensitivity_vr, 3))  + "\nDepth: " + str(depth) + " Min_split: " + str(min_split) )
  plt.plot(thresholds_vr, sensitivity_measures_vr, label="Variance Reduction")
  plt.legend(loc="upper right")

  if save_path != "":
    plt.savefig(save_path+ ".png")
  plt.show()

  plt.figure(figsize=(12,6))
  ax = plt.gca()
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)
  plt.box(on=None)
  the_table = plt.table(cellText=df.values, colLabels=df.columns,rowLabels=rows, loc='center')
  the_table.scale(1, 1.5)
  plt.subplots_adjust(left=0.2, bottom=0.4)
  if save_path != "":
    table_save_path = save_path + "-table.png"
    plt.savefig(table_save_path)
  plt.show()
  plt.show()



In [ ]:

def plot_sensitivity(data, preds, var_red_preds, depth, min_split, save_path = ""):
  use_var_red = True if var_red_preds.size else False
  use_custom = True if preds.size else False
  use_both = True if use_var_red and use_custom else False

  if use_custom:
    thresholds, sensitivity_measures, max_sensitivity, counts, s_s_list, b_s_list, max_sensitivity_threshold = get_sensitivity_values(data, preds, True)

  if use_var_red:
    thresholds_vr, sensitivity_measures_vr, max_sensitivity_vr, counts_vr, s_s_list_vr, b_s_list_vr, max_sensitivity_threshold = get_sensitivity_values(data, var_red_preds, True)

  if use_both:
    rows = ["Custom Test", "SK Test"]
    columns = ["Signal as Signal", "Signal as Background", "Background as Background", "Background as Signal"]
    table_data = {
        'w_s_s' : [counts[0], counts_vr[0]],
        'w_s_b' : [counts[1], counts_vr[1]],
        'w_b_b' : [counts[2], counts_vr[2]],
        'w_b_s' : [counts[3], counts_vr[3]],
        's_s'   : [counts[4], counts_vr[4]],
        's_b'   : [counts[5], counts_vr[5]],
        'b_b'   : [counts[6], counts_vr[6]],
        'b_s'   : [counts[7], counts_vr[7]],
        'max_sens' : [max_sensitivity, max_sensitivity_vr]
    }
  elif use_custom:
    rows = ["Custom Test"]
    columns = ["Signal as Signal", "Signal as Background", "Background as Background", "Background as Signal"]
    table_data = {
        'w_s_s' : [counts[0]],
        'w_s_b' : [counts[1]],
        'w_b_b' : [counts[2]],
        'w_b_s' : [counts[3]],
        's_s'   : [counts[4]],
        's_b'   : [counts[5]],
        'b_b'   : [counts[6]],
        'b_s'   : [counts[7]],
        'max_sens' : [max_sensitivity]
    }
  else:
    rows = ["Custom Test"]
    columns = ["Signal as Signal", "Signal as Background", "Background as Background", "Background as Signal"]
    table_data = {
        'w_s_s' : [counts_vr[0]],
        'w_s_b' : [counts_vr[1]],
        'w_b_b' : [counts_vr[2]],
        'w_b_s' : [counts_vr[3]],
        's_s'   : [counts_vr[4]],
        's_b'   : [counts_vr[5]],
        'b_b'   : [counts_vr[6]],
        'b_s'   : [counts_vr[7]],
        'max_sens' : [max_sensitivity_vr]
    }


  df = pd.DataFrame(data=table_data)
  plt.clf()
  plt.figure(figsize=(12,8))
  ylabel = "s/" +  u"\u221A" + "(s+b)"
  plt.ylabel(ylabel)
  plt.xlabel("Model Output")

  if use_both:
    plt.title("Custom Maximum Sensitivity: " +  str(round(max_sensitivity, 3))  + "\nVar-Red Maximum Sensitivity: " +  str(round(max_sensitivity_vr, 3))  + "\nDepth: " + str(depth) + " Min_split: " + str(min_split) + "\n Best threshold:" + str(max_sensitivity_threshold) )
    plt.plot(thresholds_vr, sensitivity_measures_vr, label="Var-Red Test Data")
    plt.plot(thresholds, sensitivity_measures, label="Custom Test Data")
    
  elif use_custom:
    plt.title("Custom Maximum Sensitivity: " +  str(round(max_sensitivity, 3))  + "\nDepth: " + str(depth) + " Min_split: " + str(min_split) +"\n Best threshold:" + str(max_sensitivity_threshold) )
    plt.plot(thresholds, sensitivity_measures, label="Custom Test Data")

  else:
    plt.title("Variance Reduction Maximum Sensitivity: " +  str(round(max_sensitivity_vr, 3))  + "\nDepth: " + str(depth) + " Min_split: " + str(min_split) + "\n Best threshold:" + str(max_sensitivity_threshold))
    plt.plot(thresholds_vr, sensitivity_measures_vr, label="Var-Red Test Data")

    

  plt.legend(loc="upper right")

  if save_path != "":
    plt.savefig(save_path+ ".png")
  plt.show()

  plt.figure(figsize=(12,2))
  ax = plt.gca()
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)
  plt.box(on=None)
  the_table = plt.table(cellText=df.values, colLabels=df.columns,rowLabels=rows, loc='center')
  the_table.scale(1, 1.5)
  plt.subplots_adjust(left=0.2, bottom=0.4)
  if save_path != "":
    table_save_path = save_path + "-table.png"
    plt.savefig(table_save_path)
  plt.show()

  if use_both: return round(max_sensitivity_threshold, 3), round(max_sensitivity_threshold, 3)
  elif use_var_red: return round(max_sensitivity_threshold, 3)
  return round(max_sensitivity_threshold, 3)



In [ ]:
def get_sensitivity_values(data, preds, return_counts = False):
  # compute predictions, sensitivies, true positive and false positive rates

  data['prediction'] = np.array(preds)

  signal_data = data.loc[data['data_type'] == 1]
  background_data = data.loc[data['data_type'] == 0]

  sensitivity_measures = []
  s_s_list = []
  b_s_list = []

  max_sensitivity, s_s, s_b, b_b, b_s, total, max_sensitivity_threshold = 0,0,0,0,0,0,0
  unweighted_s_s, unweighted_s_b, unweighted_b_b, unweighted_b_s = 0,0,0,0

  thresholds = np.arange(0.0, 1, 0.025)



  for threshold in thresholds:
    unweighted_signal_predicted_as_signal = signal_data.loc[signal_data["prediction"] > threshold].shape[0]
    unweighted_signal_predicted_as_background = signal_data.loc[signal_data["prediction"] <= threshold].shape[0]

    unweighted_background_predicted_as_background = background_data.loc[background_data["prediction"] <= threshold].shape[0]
    unweighted_background_predicted_as_signal = background_data.loc[background_data["prediction"] > threshold].shape[0]




    signal_predicted_as_signal = signal_data.loc[signal_data["prediction"] > threshold]['summed_weight'].sum()
    signal_predicted_as_background = signal_data.loc[signal_data["prediction"] <= threshold]['summed_weight'].sum()

    background_predicted_as_background = background_data.loc[background_data["prediction"] <= threshold]['summed_weight'].sum()
    background_predicted_as_signal = background_data.loc[background_data["prediction"] > threshold]['summed_weight'].sum()

    s_s_list.append(signal_predicted_as_signal)
    b_s_list.append(background_predicted_as_signal)


    sensitivity = 0

    if( (signal_predicted_as_signal + background_predicted_as_signal) > 0):
      sensitivity = signal_predicted_as_signal / math.sqrt((signal_predicted_as_signal + background_predicted_as_signal))

    sensitivity_measures.append(sensitivity)

    if sensitivity > max_sensitivity:
      max_sensitivity = sensitivity
      s_s = signal_predicted_as_signal
      s_b = signal_predicted_as_background
      b_b = background_predicted_as_background
      b_s = background_predicted_as_signal

      unweighted_s_s = unweighted_signal_predicted_as_signal
      unweighted_s_b = unweighted_signal_predicted_as_background
      unweighted_b_b = unweighted_background_predicted_as_background
      unweighted_b_s = unweighted_background_predicted_as_signal
      total = s_s + s_b + b_b + b_s
      max_sensitivity_threshold = threshold

  counts = [round(s_s,0), round(s_b,0), round(b_b,0), round(b_s,0), round(unweighted_s_s,0), round(unweighted_s_b,0), round(unweighted_b_b,0), round(unweighted_b_s,0) ]
  if return_counts:
    return thresholds, sensitivity_measures, round(max_sensitivity,3),counts, s_s_list, b_s_list, round(max_sensitivity_threshold, 4)
  
  return thresholds, sensitivity_measures, round(max_sensitivity,3), counts, round(max_sensitivity_threshold,4)





## Custom split selection testing 

In [ ]:
@jit(nopython= True, fastmath=True, parallel= True)
def _split_GPU(r_child, r_child_weights, l_child, l_child_weights):

  parent = np.append(l_child, r_child, axis=0)
  parent_weights = np.append(l_child_weights, r_child_weights, axis=0)

  parent_background_count = 0
  parent_signal_count = 0


  for i in range(len(parent)):
    if parent[i] == 1:
      parent_signal_count += parent_weights[i]
    else:
      parent_background_count += parent_weights[i]


  r_child_background_count = 0
  r_child_signal_count = 0

  for i in range(len(r_child)):
    if r_child[i] == 1:
      r_child_signal_count += r_child_weights[i]
    else:
      r_child_background_count += r_child_weights[i]

  measure_child = r_child_signal_count / math.sqrt( r_child_signal_count + r_child_background_count) 

  measure_parent = parent_signal_count / math.sqrt( parent_signal_count + parent_background_count)
  reduction =  measure_child - measure_parent


  return reduction


In [ ]:
def get_reduced_threshold_list(possible_thresholds, n = 10):
  try:
    return possible_thresholds[::int((len(possible_thresholds)/n))]
  except:
    return possible_thresholds

In [ ]:
def get_split_dict(n = 10):
  measure_dict = {}
  for feature in features: 
    measures =[]
    # print(feature)
    thresholds = np.sort(np.unique(df_train_valid[feature].round(decimals=3))) # thresholds
    if n != "unlimited":
      thresholds = get_reduced_threshold_list(thresholds, n = n)

    final_thresholds = []
    for threshold in thresholds:
      r_child =  df_train_valid[df_train_valid[feature] > threshold]
      if (r_child['summed_weight'].sum()> 0):
        measure= _split_GPU(r_child['data_type'].to_numpy(), r_child['summed_weight'].to_numpy(), df_train_valid['data_type'].to_numpy(), df_train_valid['summed_weight'].to_numpy())
        measures.append(measure)
        final_thresholds.append(threshold)
    measure_dict[feature] = [final_thresholds, measures]
  return measure_dict


def plot_split_dict(save_path = ""):
  for feature in features:
    print(feature, max(split_dict[feature][1]))
    thresholds= split_dict[feature][0]
    measures = split_dict[feature][1]
    plt.figure(figsize=(12,8))

    plt.title(feature)
    plt.plot(thresholds, measures)
    if save_path != "":
      path = save_path + feature +".png"
      plt.savefig(path)
    plt.show()

In [ ]:
for n in [10, 50, 100]:
    save_path = "feature_split_validation/" + str(n)
    split_dict = get_split_dict(n = n)
    plot_split_dict(save_path = save_path)

save_path = "feature_split_validation/unlimited"
split_dict = get_split_dict(n = n)
plot_split_dict(save_path = save_path)


# Plotting data

In [ ]:
def plot_multiple(data_set, label_set):
  for i in range (0, len(data_set)):
    data = data_set[i]
    label = label_set[i]
    plt.hist(data, label=label)
    plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
    plt.legend(loc="upper left")
  plt.show()

def plot_single(label):
  data = signal[label]
  plt.hist(data, label=label, bins=50)
  plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
  plt.legend(loc="upper left")
  plt.show()

def plot_signal_and_background(label):
  background_data = [item[label] for item in background]
  signal_data = signal[label]
  
  plt.rcParams['figure.figsize'] = [16, 9]
  plt.xlabel(label)
  plt.ylabel("Events per bin")
  plt.hist( background_data, bins = 100, stacked=True, label=backgroundLabel )
  # plt.hist(signal_data, bins=100, color='black', histtype=u'step', label='signal')
  plt.legend(loc='best', prop={'size': 7})
  plt.yscale('log')
  plt.show()


def scatter(x_label, y_label):
  signal_x_data, signal_y_data = signal[x_label], signal[y_label]
  background_x_data = [item[x_label] for item in background]
  background_y_data = [item[y_label] for item in background]

  plt.rcParams['figure.figsize'] = [16, 9]
  plt.rcParams["legend.markerscale"] = 3
  plt.xlabel(x_label)
  plt.ylabel(y_label)


  for i in range(0,len(background)):
    plt.scatter(background[i][x_label], background[i][y_label], s = 0.7, label = backgroundLabel[i])

  plt.scatter(signal_x_data, signal_y_data, s=1.2, color='black', label = 'signal')

  plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
  plt.legend(loc='best', prop={'size': 12})
  plt.show()

def plot_subtract(label_1, label_2):
  
  background_data = [item[label_1] - item[label_2] for item in background]
  signal_data = signal[label_1] - signal[label_2]

  plt.rcParams['figure.figsize'] = [16, 9]
  plt.xlabel(label_1 + "-" + label_2)
  plt.ylabel("Events per bin")
  plt.hist( background_data, bins = 50, stacked=True, label=backgroundLabel )
  plt.hist(signal_data, bins=50, color='black', histtype=u'step', label='signal')
  plt.legend(loc='best', prop={'size': 12})
  # plt.yscale('log')
  plt.show()

def plot_ttbar(label, save_path = "", norm=False):
  if norm:
    signal_data =  signal_norm[label]
    background_data = ttbar_norm[label]
  else:
    signal_data =  signal[label]
    background_data = ttbar[label]



  b_weights = ttbar['lumiweight'] *  ttbar['mcEventWeight']  * ttbar['pileupweight']
  s_weights = signal['lumiweight'] *  signal['mcEventWeight']  * signal['pileupweight'] 

  if norm:
    b_weights = ttbar_norm['lumiweight'] *  ttbar_norm['mcEventWeight']  * ttbar_norm['pileupweight']
    s_weights = signal_norm['lumiweight'] *  signal_norm['mcEventWeight']  * signal_norm['pileupweight'] 


  plt.rcParams['figure.figsize'] = [16, 9]
  plt.yscale('log')
  plt.xticks(fontsize=14)
  plt.yticks(fontsize=14)
  if norm:
    _, bins, p = plt.hist( background_data, bins = 50, label=backgroundLabel, weights = b_weights, color = "#003f5c", density=True)
    plt.hist(signal_data, bins=bins, color='#ffa600', label='signal (wtaunu)', weights = s_weights,  histtype=u'step', density=True)  
  else:
    _, bins, p = plt.hist( background_data, bins = 50, label=backgroundLabel, weights = b_weights, color = "#003f5c", density=True)
    plt.hist(signal_data, bins=bins, color='#ffa600', label='signal (wtaunu)', weights = s_weights,  histtype=u'step', density=True) 
     
  plt.legend(loc='best', prop={'size': 15})
  if label in mev_features: 
    plt.gca().set_xlabel(label + " [MeV]" , fontsize=18, labelpad=20)
  else: 
    plt.gca().set_xlabel(label , fontsize=18, labelpad=20) 


  if save_path:
    plt.savefig(save_path + label + ".png")
  plt.show()